In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

library(survival)
library(survminer)
library(TH.data)
library(reshape2)
library(ggplot2)
data(GBSG2)

In [ ]:
# Use help() to look at the documentation for GBSG2
help("GBSG2")

In [ ]:
head(GBSG2)


In [ ]:
# Count censored and uncensored data
print(num_cens <- table(GBSG2$cens))
# Create barplot of censored and uncensored data
barplot(table(GBSG2$cens), 
        main="Distribution of Censored vs Uncensored Data",
        xlab="Censoring Status (0=Alive, 1=Event)",
        ylab="Number of Cases",
        col=c("pink1", "pink3")) 


The presented barplot illustrates the **distribution of censored and uncensored data** in the **GBSG2** dataset.  

### What does the data mean?  
1. **Frequency table (`table(GBSG2$cens)`)**:  
   - **0** – Number of **censored cases** (alive at the end of the study).  
   - **1** – Number of **uncensored cases** (event occurred, e.g., death).  
   - Results:  
     - **387** individuals were alive (censored).  
     - **299** individuals experienced an event (uncensored).  

2. **Barplot interpretation**:  
   - Two bars represent the number of cases for categories 0 and 1.  
   - Colors:  
     - **"pink1"** for censored cases (0).  
     - **"pink3"** for uncensored cases (1).  

### Meaning of the plot:  
- More individuals (387) did not experience the event (alive) compared to those who did (299).  
- **Censoring** indicates that information about the event is unavailable at the time of analysis, which is common in **Survival Analysis**.  



In [ ]:
# Check the structure of the dataset
str(GBSG2)


In [ ]:
# Summary statistics for all variables
summary(GBSG2)

In [ ]:
# Distribution of survival times
hist(GBSG2$time, main="Distribution of Survival Times", xlab="Time", col="pink1")

## Charakterystyka rozkładu
### Kształt
Rozkład czasów przeżycia jest wielomodalny, z głównym szczytem występującym około 500 dni oraz mniejszymi lokalnymi maksimami w okolicy 1500-2000 dni1
.
### Częstości
Najwyższa częstość występuje w przedziale około 500 dni, osiągając wartość około 90 przypadków1
Średnie częstości (40-60 przypadków) utrzymują się w przedziale 1000-2000 dni1
Najniższe częstości występują w końcowym okresie obserwacji (powyżej 2000 dni)1
### Zakres danych
Czasy przeżycia rozciągają się od 0 do około 2500 dni, przy czym większość obserwacji koncentruje się w pierwszych 2000 dniach
.

In [ ]:
# Create Surv-Object
sobj <- Surv(GBSG2$time, GBSG2$cens)

# Look at 10 first elements
sobj[1:10]

# Look at summary
summary(sobj)

# Look at structure
str(sobj)


### What does **`Surv()`** change?  
- **Combines `time` and `cens`** into a single object that maintains the relationship between these two variables.  
- **Adds information about censoring type** (e.g., **"right"**) and displays a **`+` symbol** for censored observations.  
- **Creates a format required for survival models**, such as **Kaplan-Meier** and **Cox Proportional-Hazards**.

In [ ]:
# Correlation matrix for numerical variables
cor(GBSG2[, sapply(GBSG2, is.numeric)])

# Check the distribution of categorical variables
lapply(GBSG2[, sapply(GBSG2, is.factor)], table)

In [ ]:


# Correlation matrix for numerical variables
cor(GBSG2[, sapply(GBSG2, is.numeric)])

# Check the distribution of categorical variables
lapply(GBSG2[, sapply(GBSG2, is.factor)], table)


# Check for outliers in numerical variables
boxplot(GBSG2[, sapply(GBSG2, is.numeric)], main="Boxplots for Numerical Variables")

# These additional checks will provide a more comprehensive understanding of the dataset,
# including its structure, potential issues with missing data or outliers,
# and initial insights into survival patterns.


In [ ]:
library(reshape2)
library(ggplot2)

# Compute the correlation matrix
cor_matrix <- cor(GBSG2[, sapply(GBSG2, is.numeric)])

# Reshape the correlation matrix into long format
melted_cor <- melt(cor_matrix)

# Create the heatmap with larger text and size adjustments
ggplot(data = melted_cor, aes(Var1, Var2, fill = value)) +
  geom_tile() +                                   # Create heatmap tiles
  scale_fill_gradient2(low = "white", high = "red", mid = "pink", 
                       midpoint = 0, limit = c(-1, 1), space = "Lab") + # Gradient colors
  theme_minimal(base_size = 15) +                 # Increase base font size for the plot
  theme(
    axis.text.x = element_text(angle = 45, vjust = 1, hjust = 1, size = 18), # Larger x-axis text
    axis.text.y = element_text(size = 18),                                  # Larger y-axis text
    axis.title.x = element_blank(),                                         # Remove x-axis title
    axis.title.y = element_blank(),                                         # Remove y-axis title
    legend.title = element_text(size = 14),                                 # Larger legend title
    legend.text = element_text(size = 12)                                   # Larger legend labels
  ) +
  labs(title = "Correlation Heatmap", fill = "Correlation") + # Add plot title and legend label
  coord_fixed() # Fix aspect ratio to make tiles square


In [ ]:
# Load necessary libraries
library(survival)
library(survminer)

# Kaplan-Meier survival estimate
km <- survfit(Surv(time, cens) ~ 1, data = GBSG2)

# Plot the survival curve with additional options
ggsurvplot(fit = km, 
           palette = "pink",              
           linetype = 1,                     # Use a solid line style
           surv.median.line = "hv",          # Add median survival lines (horizontal and vertical)
           risk.table = TRUE,                # Add risk table below the plot
           cumevents = TRUE,                 # Show cumulative number of events
           cumcensor = TRUE,                 # Show cumulative number of censored observations
           tables.height = 0.1,              # Adjust height of tables
           tables.theme = theme_survminer(font.size = 4)) # Set font size for tables


In [ ]:
# Weibull model
wb <- survreg(Surv(time, cens) ~ 1, data = GBSG2)

# 70 Percent of patients survive beyond time point...
predict(wb, type = "quantile", p = 1-0.7, newdata = data.frame(1))

70 out of 100 patients survive more than 1004.524 days.

In [ ]:
# Retrieve survival curve from model probabilities 
surv <- seq(.99, .01, by = -.01)  # Survival probabilities from 99% to 1%

Creates a sequence of survival probabilities from 99% to 1%, decreasing by 1% at each step.
Used to estimate times corresponding to each probability.

In [ ]:
t <- predict(wb, type = "quantile", p = 1 - surv, newdata = data.frame(1))
surv_wb <- data.frame(time = t, surv = surv)
head(surv_wb)


In [ ]:
plot(surv_wb$time, surv_wb$surv, type = "l", col = "blue",
     xlab = "Time", ylab = "Survival Probability", main = "Weibull Survival Curve")


In [ ]:
library(survminer)
wbmod <- survreg(Surv(time, cens) ~ horTh, data = GBSG2)
coef(wbmod)

# Retrieve survival curve from model
surv <- seq(.99, .01, by = -.01)
t_yes <- predict(wbmod, type = "quantile", p =1 - surv,
    newdata = data.frame(horTh = "yes"))

# Take a look at survival curve
str(t_yes)


Each row of t corresponds to one covariate combination (one imaginary patient) and each column to one value of surv

In [ ]:
# Weibull model
wbmod <- survreg(Surv(time, cens) ~ horTh + tsize, data = GBSG2, dist = "weibull")

# Imaginary patients
newdat <- expand.grid(
  horTh = levels(GBSG2$horTh), # Two levels of the covariate 'horTh'
  tsize = quantile(GBSG2$tsize, probs = c(0.25, 0.50, 0.75)) # 25%, 50%, 75% quantiles for 'tsize'
)

# View the new data frame
newdat


Combines time and survival probabilities into a data frame.
Adds placeholders for confidence intervals (upper, lower) and standard error (std.err), which are not computed here but required by ggsurvplot_df(). A smooth survival curve based on the Weibull model showing how survival probability decreases over time. Confidence intervals (upper, lower) are set to NA because parametric confidence intervals are not included in this example.

In [ ]:
#library(survminer)
#library(survival)


# Weibull model
wb <- survreg(Surv(time, cens) ~ 1, data = GBSG2)

# Retrieve survival curve from model
surv <- seq(.99, .01, by = -.01)

# Get time for each probability

t <- predict(wb, type ="quantile", p =1- surv, newdata = data.frame(1))

# Create data frame with the information needed for ggsurvplot_df
surv_wb <- data.frame(time = t, surv = surv,                      upper =NA, lower =NA, std.err =NA)
ggsurvplot_df(fit = surv_wb, surv.geom = geom_line)


In [ ]:
# Weibull model
wbmod <- survreg(Surv(time, cens) ~ horTh + tsize, data = GBSG2)

# Imaginary patients
newdat <- expand.grid(
  horTh = levels(GBSG2$horTh),
  tsize = quantile(GBSG2$tsize, probs = c(0.25, 0.5, 0.75)))

# Compute survival curves
# Compute survival curves
surv <- seq(.99, .01, by = -.01)
t <- predict(wbmod, type = "quantile", p = 1 - surv,
  newdata = newdat)

# How many rows and columns does t have?
dim(t)

In [ ]:
# Combine newdat with predicted survival times
surv_wbmod_wide <- cbind(newdat, t)

# Reshape to long format using melt
library(reshape2) # Make sure reshape2 is loaded
surv_wbmod <- melt(surv_wbmod_wide, 
                   id.vars = c("horTh", "tsize"), # Variables to keep
                   variable.name = "surv_id",    # Column name for melted variable IDs
                   value.name = "time")         # Column name for melted values
library(survminer)

# Use surv_wbmod$surv_id to add the correct survival probabilities surv
surv_wbmod$surv <- surv[as.numeric(surv_wbmod$surv_id)]

# Add columns upper, lower, std.err, and strata to the data.frame
surv_wbmod[, c("upper", "lower", "std.err", "strata")] <- NA

# Plot the survival curves
ggsurvplot_df(surv_wbmod, surv.geom = geom_line,
  linetype = "horTh", color = "tsize", legend.title = NULL)


The visualization shows that patients with smaller tumors tend to survive longer and patients who receive hormonal therapy tend to survive longer.

geom_line will produce a smooth line correctly representing the survival curve estimate from the Weibull model (the default surv.geom = geom_step shows a step function and is incorrect for Weibull models

In [ ]:
survreg(Surv(time, cens)~ horTh,   data = GBSG2)
survreg(Surv(time, cens)~ horTh,   data = GBSG2,   dist ="exponential")
survreg(Surv(time, cens)~ horTh,   data = GBSG2,   dist ="lognormal")

In [ ]:
# Weibull model
wbmod <- survreg(Surv(time, cens) ~ horTh + tsize, data = GBSG2, dist = "weibull")

# Log-Normal model
lnmod <- survreg(Surv(time, cens) ~ horTh + tsize, data = GBSG2, dist = "lognormal")

# Define new data (imaginary patients)
newdat <- expand.grid(
  horTh = levels(GBSG2$horTh),                          # Hormonal therapy levels
  tsize = quantile(GBSG2$tsize, probs = c(0.25, 0.5, 0.75)) # Tumor size quantiles
)

# Survival probabilities
surv <- seq(.99, .01, by = -0.01)

# Compute survival times from Weibull model
wbt <- predict(wbmod, type = "quantile", p = 1 - surv, newdata = newdat)

# Compute survival times from Log-Normal model
lnt <- predict(lnmod, type = "quantile", p = 1 - surv, newdata = newdat)

# Check dimensions
dim(wbt)
dim(lnt)


In [ ]:
 #6. Combine predictions into a wide format data frame
surv_wide <- cbind(
  newdat,
  dist = rep(c("Weibull", "Log-Normal"), each = nrow(newdat)),
  rbind(wbt, lnt)  # Combine predictions for both models
)

# 7. Melt the data.frame into long format
surv_long <- melt(surv_wide,
                  id.vars = c("horTh", "tsize", "dist"),  # Keep identifiers
                  variable.name = "surv_id",             # Name for melted variable
                  value.name = "time")                   # Name for time values

# 8. Add survival probabilities
surv_long$surv <- surv[as.numeric(gsub("V", "", surv_long$surv_id))]

# 9. Add required columns with NA values
surv_long[, c("upper", "lower", "std.err", "strata")] <- NA

# 10. Plot the survival curves
library(survminer)
ggsurvplot_df(surv_long, surv.geom = geom_line,
              linetype = "horTh", color = "dist", legend.title = NULL)


Comparing Weibull and Log-Normal Model I

In [ ]:
# Cox model
cxmod <- coxph(Surv(time, cens) ~ horTh + tsize, data = GBSG2)

# Imaginary patients
newdat <- expand.grid(
  horTh = levels(GBSG2$horTh),
  tsize = quantile(GBSG2$tsize, probs = c(0.25, 0.5, 0.75)))
rownames(newdat) <- letters[1:6]

# Inspect newdat
newdat

# Compute survival curves
cxsf <- survfit(cxmod, data = GBSG2, newdata = newdat, conf.type = "none")
# Look at first 6 rows of cxsf$surv and time points
head(cxsf$surv)
head(cxsf$time)

surv_wide is a wide data frame containing hormonal therapy information and the survival curves for the Weibull and log-normal models.  the survival probability is more drawn out for the log-normal distribution than the Weibull distribution

In [ ]:
# Compute survival curves
cxsf <- survfit(cxmod, data = GBSG2, newdata = newdat, conf.type = "none")
# Look at first 6 rows of cxsf$surv and time points
head(cxsf$surv)
head(cxsf$time)